In [1]:
import pandas as pd
import geopandas as gpd
import json
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

# Load data and combine
1. Load Aleph wind farms
2. Load MapStand data
3. Combine
4. Save

## Load Aleph Data
need to add content of ftm  env to current environment for this to work


In [2]:
host = "https://data.ftm.nl/"
api_key = "Ai9e2w9tc95-JsvrbGQSX4moHgriPD2nm3lo9TogC_k"
foreign_id = "ce5001727ffb4d3cb42cc8c594395c22"
fn = r"C:\Users\n649505\code\ftm_north_sea_data\north_sea_wind_assets.csv"
#os.system("alephclient --host %s --api-key %s stream-entities -f %s -s Asset > %s" % (host, api_key, foreign_id, fn))
print("alephclient --host %s --api-key %s stream-entities -f %s -s Asset > %s" % (host, api_key, foreign_id, fn))

alephclient --host https://data.ftm.nl/ --api-key Ai9e2w9tc95-JsvrbGQSX4moHgriPD2nm3lo9TogC_k stream-entities -f ce5001727ffb4d3cb42cc8c594395c22 -s Asset > C:\Users\n649505\code\ftm_north_sea_data\north_sea_wind_assets.csv


Saved format from Aleph is funky. Open in MS Excel and save as csv

In [3]:
data_fn = "../../../ftm_north_sea_data/north_sea_wind_assets.csv"

raw_df = pd.read_csv(data_fn,header=None, encoding="utf-8")

asset_df = pd.json_normalize([json.loads(line) for line in raw_df[0].tolist() if json.loads(line)["schema"]=="Asset"])

asset_df.rename(columns={"properties.name":"name", "properties.notes": "status", "properties.country": "country", "properties.description": "mps_id", 
                         "properties.amountEur": "value_eur", "properties.publisherUrl": "source", "properties.amount": "installed_capacity_mw",
                         "id": "aleph_id", "properties.createdAt": "aleph_year"}, inplace=True)

cols = ["name", "status", "installed_capacity_mw", "country", "mps_id", "value_eur", "source", "aleph_year"]
for col in cols:
    asset_df[col] = asset_df[col].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

print("Aleph dataset has %d windfarms" % asset_df.shape[0])
asset_df.head()

Aleph dataset has 418 windfarms


,schema,aleph_id,name,properties.alias,status,country,mps_id,source,properties.alephUrl,properties.proof,properties.summary,installed_capacity_mw,value_eur,aleph_year
0,Asset,335b0a1c-6a1d-4852-861c-350ced054588.8a775aaa5...,ERO FLOATING WIND TURBINE,[ÆRØ FORSØGSMØLLE],EXCLUSIVE_DEVELOPMENT_RIGHTS,dk,2d6f892c-21db-464f-a653-fade44071fcf,https://www.mapstand.com/,[https://data.ftm.nl/api/2/entities/335b0a1c-6...,NaN,NaN,NaN,NaN,NaN
1,Asset,31c8264d101bd43e14f989e15ff4979d861b87fd.23453...,BORNHOLM I SOUTH,NaN,AREA_UNDER_STATE_INVESTIGATION,dk,fca6a2b3-283d-4846-84d0-d7aad9bc4d7f,NaN,[https://data.ftm.nl/api/2/entities/31c8264d10...,[307482.7f01693766e3a0b61e86179f5ff48b87a83194ac],[Windfarm development site that together with ...,NaN,NaN,NaN
2,Asset,3bc15b25b0162ace8ba03e60903f83489c6b95fc.6d02a...,POST 2030 WIND AREA 6 (ADDITIONAL),NaN,AREA_UNDER_STATE_INVESTIGATION,nl,174cb7c6-7c96-47f8-9bcf-2a2c58a228e9,NaN,[https://data.ftm.nl/api/2/entities/3bc15b25b0...,[307482.7f01693766e3a0b61e86179f5ff48b87a83194ac],[This is an additional search area for wind en...,NaN,NaN,NaN
3,Asset,4322eac06344a986e8f2b519333cd7bfd06f63df.15c9d...,NORDSOEN TENDER 6,NaN,AREA_UNDER_STATE_INVESTIGATION,dk,23905ea7-cb54-4873-8d56-d8e44cf5f230,NaN,[https://data.ftm.nl/api/2/entities/4322eac063...,[307482.7f01693766e3a0b61e86179f5ff48b87a83194ac],[Location of a fixed-bottom wind tender area],NaN,NaN,NaN
4,Asset,489a798fdf0af862c5390fa59f9cd1ecf8fe0b7c.0c716...,NORDSOEN TENDER 4,NaN,AREA_UNDER_STATE_INVESTIGATION,dk,7fd2c770-74e6-4902-a973-e0208215bd48,NaN,[https://data.ftm.nl/api/2/entities/489a798fdf...,[307482.7f01693766e3a0b61e86179f5ff48b87a83194ac],[Location of a fixed-bottom wind tender area],NaN,NaN,NaN


## Load MapStand

In [4]:
token_wfs = "0f5ed330-90f4-4f28-9214-5d981f3db642"
token_wms = "74d99a64-fdf4-4641-a390-c909665414ff"

url = "https://hub.mapstand.com/gs/ows?service=WFS&VERSION=1.0.0&request=GetFeature&typename=mps:powerplant_windfarm_installed&outputFormat=json&srsName=EPSG:4326&apikey=%s" % token_wms
response = requests.get(url)
data = response.json()["features"]
gdf_installed = gpd.GeoDataFrame.from_features(data, crs=4326)

url = "https://hub.mapstand.com/gs/ows?service=WFS&VERSION=1.0.0&request=GetFeature&typename=mps:powerplant_windfarm_planned&outputFormat=json&srsName=EPSG:4326&apikey=%s" % token_wms
response = requests.get(url)
data = response.json()["features"]
gdf_planned = gpd.GeoDataFrame.from_features(data, crs=4326)

url = "https://hub.mapstand.com/gs/ows?service=WFS&VERSION=1.0.0&request=GetFeature&typename=mps:wind_development_zone&outputFormat=json&srsName=EPSG:4326&apikey=%s" % token_wms
response = requests.get(url)
data = response.json()["features"]
gdf_wdz = gpd.GeoDataFrame.from_features(data, crs=4326)

#concatenate
rdf = gpd.GeoDataFrame(pd.concat((gdf_planned, gdf_installed, gdf_wdz), ignore_index=True), crs=gdf_planned.crs)

## Combine datasets
1. Use capacity from Aleph if it exists 
2. Add installation year from Aleph if it exists `aleph_year`
3. Make a simplified status column
4. Remove Norwegian competing tenders
5. Subselect columns
6. Save to geojson (for streamlit app) and excel 

In [5]:
cols_to_keep = ["mps_uuid", "capacity_mw", "geometry", "installation_year", 
                "mps_est_elevation_min_m", "mps_est_elevation_max_m", "coast_distance",
                "mps_est_shore_status", "windfarm_type", "number_generators",
                "mps_est_area_sqkm"]
merged = pd.merge(asset_df, rdf[cols_to_keep],
          left_on = "mps_id", right_on="mps_uuid", how="left").set_index("aleph_id")

# update capacities with manually entered ones
merged["capacity"] = pd.to_numeric(merged.installed_capacity_mw.apply(lambda x: x.strip(" MW") if isinstance(x, str) else x)).fillna(merged.capacity_mw)

# correct installation year
merged["installation_year"] = merged.installation_year.fillna(merged["aleph_year"])

In [9]:
# status dictionary
st_dict = {'OPERATIONAL': "EXISTING", 
 'EXCLUSIVE_DEVELOPMENT_RIGHTS': "EARLY_STAGE_PLANS",
 'UNDER_CONSTRUCTION': "EXISTING", 
 'PROJECT_ANNOUNCED': "POTENTIAL_PLANS", 
 'EARLY_PLANNING': "EARLY_STAGE_PLANS",
'APPLICATION_SUBMITTED': "LATE_STAGE_PLANS", 
'CONSENT_AUTHORISED': "LATE_STAGE_PLANS", 
'EXTENSION_REQUESTED': "LATE_STAGE_PLANS", 
'CANCELLED': "", 
'ONSHORE - OPERATIONAL': "",
'DECOMMISSIONED': "", 
'AREA_PROPOSED': "POTENTIAL_PLANS",
"AREA_UNDER_STATE_INVESTIGATION": "POTENTIAL_PLANS",
"COMPETING_PROPOSALS": "EARLY_STAGE_PLANS",
"TENDER_PENDING": "EARLY_STAGE_PLANS",
"PLANNING_PERMISSION_EXPIRED": ""}
merged["simple_status"] = merged.status.replace(st_dict)

In [10]:
cols_to_keep = ['name', 'status','simple_status', 'country', 'value_eur', 
        'capacity', 'installation_year', "coast_distance", "number_generators", 'mps_est_elevation_min_m',
       'mps_est_shore_status', 'windfarm_type', 'mps_est_area_sqkm',
       'source']

merged[cols_to_keep].to_excel("../../../ftm_north_sea_data/north_sea_windparks_v6.xlsx")

In [11]:
cols_to_keep = ['name', 'status', 'country', 'value_eur',"number_generators",
       'geometry', 'installation_year', 'mps_est_elevation_min_m', "mps_est_elevation_max_m",
       'mps_est_shore_status', 'windfarm_type', 'mps_est_area_sqkm',
       'capacity', 'simple_status']
df = gpd.GeoDataFrame(merged[cols_to_keep], geometry=merged.geometry, crs="EPSG:4326")

df.to_file("../../../ftm_north_sea_data/wind_farms_all_%s.geojson" % datetime.datetime.now().strftime('%Y%m%d'), driver="GeoJSON")

### Clean Norway's data

In [12]:
# remove Norway's announced projects as they are redundant with the area proposed
no_redundant_plans = ["a89d6541-e184-49bb-9523-074760f8e205", # Skjoldblad 1
                      "a71d7529-9523-4339-8ecb-1d9c38304a5d", #Vestavindar
                      "b1691a7e-c118-4202-8743-220db5f5964a", # Sonnavindar 1
                      "11808e1a-00eb-4ff3-a243-0e2fbae33948", # not in data anymore
                      "0cea04d5-5e6b-444c-967d-3eaf0141c8e9", # Skjoldblad 2
                      "1a142c99-525d-4a53-a3a9-229717c372b2"] # Sonnavindar 2

merged_no = merged.drop(merged[merged.mps_id.isin(no_redundant_plans)].index)

# alternatively drop all the plans with status competing proposals
merged[(merged.country=="no") & (merged.status=="COMPETING_PROPOSALS")]

,schema,name,properties.alias,status,country,mps_id,source,properties.alephUrl,properties.proof,properties.summary,...,installation_year,mps_est_elevation_min_m,mps_est_elevation_max_m,coast_distance,mps_est_shore_status,windfarm_type,number_generators,mps_est_area_sqkm,capacity,simple_status
aleph_id,,,,,,,,,,,,,,,,,,,,,
a33801b3-8d21-48be-8803-13daa96089fa.c083dd875bf7b18b29db7f74e98bd3195ec725ac,Asset,SOUTHERN NORTH SEA II (PHASE 2) - SKJOLDBLAD,NaN,COMPETING_PROPOSALS,no,0cea04d5-5e6b-444c-967d-3eaf0141c8e9,https://www.mapstand.com/,[https://data.ftm.nl/api/2/entities/a33801b3-8...,NaN,NaN,...,2030.0,-75.0,-50.0,NaN,OFFSHORE,OFFSHORE_FIXED,NaN,1264.613853,1500.0,EARLY_STAGE_PLANS
828ab8ce-7825-4cdf-9ff6-f16a66fc66c8.02e1996aaed40610ed6537ef1f42e6c5dec68583,Asset,VESTAVINDAR,NaN,COMPETING_PROPOSALS,no,a71d7529-9523-4339-8ecb-1d9c38304a5d,https://www.mapstand.com/,[https://data.ftm.nl/api/2/entities/828ab8ce-7...,NaN,NaN,...,2026.0,-283.0,-191.0,NaN,OFFSHORE,OFFSHORE_FLOATING,NaN,1001.768390,1500.0,EARLY_STAGE_PLANS
f4c64c05-af5a-4d3e-8b36-ab2791e25671.3114e066ac5c899d04f1a256a58f31872fb630e4,Asset,SONNAVINDAR PHASE 2,NaN,COMPETING_PROPOSALS,no,1a142c99-525d-4a53-a3a9-229717c372b2,https://www.mapstand.com/,[https://data.ftm.nl/api/2/entities/f4c64c05-a...,NaN,NaN,...,2029.0,-75.0,-50.0,NaN,OFFSHORE,OFFSHORE_MIXED,NaN,1264.613853,1500.0,EARLY_STAGE_PLANS
f682b067-1902-4cad-8696-7a8fc38a0ad4.83b0194d227b46af3b424d3b20d904401a214f40,Asset,SONNAVINDAR PHASE 1,NaN,COMPETING_PROPOSALS,no,b1691a7e-c118-4202-8743-220db5f5964a,https://www.mapstand.com/,[https://data.ftm.nl/api/2/entities/f682b067-1...,NaN,NaN,...,2026.0,-85.0,-50.0,NaN,OFFSHORE,OFFSHORE_MIXED,NaN,1300.877540,1500.0,EARLY_STAGE_PLANS
12e08f7a-fa9c-4af7-9d1f-71e53866bb1e.eacb689005d0342f146f7cd8b672cc1735333e95,Asset,SOUTHERN NORTH SEA II (PHASE 1) - SKJOLDBLAD,NaN,COMPETING_PROPOSALS,no,a89d6541-e184-49bb-9523-074760f8e205,https://www.mapstand.com/,[https://data.ftm.nl/api/2/entities/12e08f7a-f...,NaN,NaN,...,2030.0,-85.0,-50.0,NaN,OFFSHORE,OFFSHORE_FIXED,NaN,1300.877540,1500.0,EARLY_STAGE_PLANS


### Save data to geojson

In [13]:
cols_to_keep = ['name', 'status', 'country', 'value_eur',
       'mps_id', 'source', 'installed_capacity_mw',
        'mps_uuid', 'capacity_mw', "number_generators",
       'geometry', 'installation_year', 'mps_est_elevation_min_m', "mps_est_elevation_max_m",
       'mps_est_shore_status', 'windfarm_type', 'mps_est_area_sqkm',
       'capacity', 'simple_status']
df = gpd.GeoDataFrame(merged_no[cols_to_keep], geometry=merged_no.geometry, crs="EPSG:4326")
df.to_file("../../../ftm_north_sea_data/wind_farms_%s.geojson" % datetime.datetime.now().strftime('%Y%m%d'), driver="GeoJSON")
